In [1]:
import pickle
import numpy as np
import tensorflow as tf
import math

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
    test = unpickle(pos + '/test_batch')
    Xte = test[b'data']
    Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.eye(10)[Ytr]
    Yte_onehot = np.eye(10)[Yte]
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")

# label data of train and test data, label data is represented by one-hot encoding
Ytr, Yte = onehot_encoding(Ytr, Yte)

In [38]:
class CNN:
    def __init__(self, sess, beta, gamma, n_row):
        self.sess = sess
        self.beta = beta
        self.gamma = gamma
        self.batch_size = None
        self.n_row = n_row
        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.Y = tf.placeholder(tf.float32, [None, 10])
    def modeling(self):
#         X_, y_ = self.X, self.Y
        L1 = self.Layer(self.X, 7, 3, 64, 2, 'Layer_1')
            # Tensor("Layer_1_15/Relu:0", shape=(?, 16, 16, 64), dtype=float32)
        L1_identity = self.pooling(L1, 3, 2)
            # Tensor("MaxPool_28:0", shape=(?, 8, 8, 64), dtype=float32)
        L2_1 = self.Layer(L1_identity, 1, 64, 64, 1, 'Layer_2_1', pad='VALID')
            # Tensor("Layer_2_1_14/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
        L2_2 = self.Layer(L2_1, 3, 64, 64, 1, 'Layer_2_2', pad='VALID')
            # Tensor("Layer_2_2_14/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
        L2_3 = self.Layer(L2_2, 1, 64, 256, 1, 'Layer_2_3', pad='VALID') 
            # Tensor("Layer_2_3_14/Relu:0", shape=(?, 8, 8, 256), dtype=float32)
        L2_4 = self.Layer(L1_identity, 1, 64, 256, 1, 'Layer_2_4', pad='VALID')
            # Tensor("Layer_2_4_14/Relu:0", shape=(?, 8, 8, 256), dtype=float32)
        L2 = tf.nn.relu(L2_3 + L2_4) # Tensor("Relu_14:0", shape=(?, 8, 8, 256), dtype=float32)
        
        Layer = self.pooling(L2, 8, 1) # Tensor("MaxPool_29:0", shape=(?, 8, 8, 256), dtype=float32)
        
        Layer = tf.reshape(Layer, [-1, 256]) # Tensor("Reshape_56:0", shape=(?, 256), dtype=float32)
        
        W = tf.Variable(tf.random_normal([256, 10], stddev=0.01), name='fully-connected')
        self.Layer = tf.matmul(Layer, W)
        self.loss_funtion(self.Y)

    def _batch(self, iterable, length, n=1):
        for ndx in range(0, length, n):
            yield iterable[ndx:min(ndx + n, length)]
    def train(self, X, y, n_epoch, batch_size):
        n_row = self.n_row
        sess = self.sess
        self.batch_size = batch_size
#         n_step = n_row // batch_size
#         X_ = tf.placeholder(tf.float32, [None, 32, 32, 3])
#         y_ = tf.placeholder(tf.float32, [None, 10])
#         X_, y_ = self.X, self.Y
#         self.loss_funtion(y_)
#         loss = self.loss
#         optimizer = self.optimizer
        for epoch in range(n_epoch):
            arr = np.random.randint(n_row, size=n_row)
            for x in self._batch(range(n_row), n_row, batch_size):
                batch_X, batch_y = X[arr[x], :], y[arr[x]]
                loss_, _ = sess.run([self.loss, self.optimizer], feed_dict={self.X:batch_X, self.Y:batch_y})
                print(loss_)
    def loss_funtion(self, y, learning_rate=0.001):
        Layer = self.Layer
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Layer, labels=y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
        self.loss = loss
        
    def Layer(self, X, filter_size, in_feature, out_feature, stride, name, pad = 'SAME'):
        with tf.variable_scope(name):
            L = self.conv_bn_relu(X, filter_size, in_feature, out_feature, stride, pad = 'SAME')
        return L
    def conv_bn_relu(self, X, filter_size, in_feature, out_feature, stride, pad = 'SAME'):
        batch, beta, gamma = self.batch_size, self.beta, self.gamma
        stddev = math.sqrt(2/batch) if batch else 0.01
        W = tf.Variable(tf.random_normal([filter_size, filter_size, in_feature, out_feature], stddev=stddev))
        L = tf.nn.conv2d(X, W, strides=[1,stride,stride,1], padding=pad)

        L = tf.nn.batch_normalization(L, offset=beta, scale=gamma, mean=0.001, variance=0.001, variance_epsilon=1e-5)
        L = tf.nn.relu(L)
        return L
    def pooling(self, L, ksize, stride):
        L = tf.nn.max_pool(L, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding='SAME')
        return L
    def save_model(self):
        pass

In [51]:
### Implement the layers of CNNs ###

In [52]:
# cost function, you can change the implementation
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
# optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [39]:
# initialize the variables
# init = tf.global_variables_initializer()
# sess = tf.Session()
# sess.run(init)
beta, gamma = 0.5, 0.5
n_row, batch_size = 50000, 100
n_epoch = 10
with tf.Session() as sess:
    res = CNN(sess, beta, gamma, n_row)
    res.modeling()
    sess.run(tf.global_variables_initializer())
    res.train(Xtr, Ytr, n_epoch, batch_size)

Tensor("Layer_1_15/Relu:0", shape=(?, 16, 16, 64), dtype=float32)
Tensor("MaxPool_28:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("Layer_2_1_14/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("Layer_2_2_14/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("Layer_2_3_14/Relu:0", shape=(?, 8, 8, 256), dtype=float32)
Tensor("Layer_2_4_14/Relu:0", shape=(?, 8, 8, 256), dtype=float32)
Tensor("Relu_14:0", shape=(?, 8, 8, 256), dtype=float32)
Tensor("MaxPool_29:0", shape=(?, 8, 8, 256), dtype=float32)
Tensor("Reshape_56:0", shape=(?, 256), dtype=float32)


InvalidArgumentError: logits and labels must be same size: logits_size=[6400,10] labels_size=[100,10]
	 [[Node: SoftmaxCrossEntropyWithLogits_14 = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape_57, Reshape_58)]]

Caused by op 'SoftmaxCrossEntropyWithLogits_14', defined at:
  File "C:\Users\yun\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\yun\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\yun\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\yun\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\yun\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  Fil

In [54]:
### Implement the train process ###

In [55]:
### Implement the test process ###